# NORMALIZING DATA

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine # read_sql_table only works with sqlalchemy connection
import io

In [2]:
username = 'postgres'
password = 'infint'
host = 'localhost'
port = '5432'
database = 'postgres'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, database))

In [3]:
def df_to_sql(df, name, engine, if_exists='replace'):
    df.head(0).to_sql(name, engine, if_exists=if_exists, index=False)
    connection = engine.raw_connection()
    cursor = connection.cursor()
    output = io.StringIO()
    df.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    contens = output.getvalue()
    cursor.copy_from(output, name, null="")
    connection.commit()

## IHME-GBD DATA

In [4]:
# create lookup tables
measure_names = {}
location_names = {}
sex_names = {}
age_names = {}
cause_names = {}
metric_names = {}

lookups = [
    ['measure', {}],
    ['location', {}],
    ['sex', {}],
    ['age', {}],
    ['cause', {}],
    ['metric', {}],
]

In [5]:
sql = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'"

In [6]:
for row in engine.execute(sql):
    table_name = row[0]
    if table_name.startswith('ihme_gbd'):
        print(table_name)
        df = pd.read_sql_table(table_name, con=engine)
        
        # create the lookup tables by finding the key value pairs for measures, locations, ...
        for lookup_idx, (lkp_key, kv_store) in enumerate(lookups):
            for _name in df['{}_name'.format(lkp_key)].unique():
                _id = df['{}_id'.format(lkp_key)].loc[df['{}_name'.format(lkp_key)] == _name].unique()
                if len(_id) != 1:
                    print("No unique id for {}: {}".format(lkp_key, _name))
                else:
                    lookups[lookup_idx][1][_id[0]] = _name

ihme_gbd_2017_data_cb71683c_27
ihme_gbd_2017_data_cb71683c_28
ihme_gbd_2017_data_cb71683c_29
ihme_gbd_2017_data_cb71683c_3
No unique id for location: North Africa and Middle East
ihme_gbd_2017_data_cb71683c_30
ihme_gbd_2017_data_cb71683c_31
ihme_gbd_2017_data_cb71683c_4
ihme_gbd_2017_data_cb71683c_5
No unique id for location: North Africa and Middle East
No unique id for location: Georgia
No unique id for location: South Asia
ihme_gbd_2017_data_cb71683c_6
No unique id for location: Georgia
No unique id for location: North Africa and Middle East
No unique id for location: South Asia
ihme_gbd_2017_data_cb71683c_7
No unique id for location: Georgia
ihme_gbd_2017_data_cb71683c_8
No unique id for location: Georgia
ihme_gbd_2017_data_cb71683c_9
No unique id for location: South Asia
No unique id for location: Georgia
ihme_gbd_2017_data_cb71683c_1
No unique id for location: North Africa and Middle East
No unique id for location: South Asia
No unique id for location: Georgia
ihme_gbd_2017_data_

In [7]:
for kv_store in lookups:
    metric, dict_values = kv_store
    columns = ['key', 'value']
    values = []
    for key, val in dict_values.items():
        values.append([key, val])
    metric_df = pd.DataFrame(data=values, columns=columns)
    metric_df.to_sql(metric, con=engine, if_exists='replace')

In [8]:
for row in engine.execute(sql):
    table_name = row[0]
    if table_name.startswith('ihme_gbd'):
        print(table_name)
        df = pd.read_sql_table(table_name, con=engine)
        
        for lkp_key, kv_store in lookups:
            df.drop('{}_name'.format(lkp_key), axis=1, inplace=True)
            print("dropped {}_name".format(lkp_key))
        df_to_sql(df, '{}'.format(table_name), engine, if_exists='replace')
        print("stored table")

ihme_gbd_2017_data_cb71683c_27
dropped measure_name
dropped location_name
dropped sex_name
dropped age_name
dropped cause_name
dropped metric_name
stored table
ihme_gbd_2017_data_cb71683c_28
dropped measure_name
dropped location_name
dropped sex_name
dropped age_name
dropped cause_name
dropped metric_name
stored table
ihme_gbd_2017_data_cb71683c_29
dropped measure_name
dropped location_name
dropped sex_name
dropped age_name
dropped cause_name
dropped metric_name
stored table
ihme_gbd_2017_data_cb71683c_3
dropped measure_name
dropped location_name
dropped sex_name
dropped age_name
dropped cause_name
dropped metric_name
stored table
ihme_gbd_2017_data_cb71683c_30
dropped measure_name
dropped location_name
dropped sex_name
dropped age_name
dropped cause_name
dropped metric_name
stored table
ihme_gbd_2017_data_cb71683c_31
dropped measure_name
dropped location_name
dropped sex_name
dropped age_name
dropped cause_name
dropped metric_name
stored table
ihme_gbd_2017_data_cb71683c_4
dropped mea

## WDI Data

### WDI Footnote

In [23]:
df = pd.read_sql_table('wdifootnote', con=engine)

In [24]:
mappings_description = {}
descriptions = list(df['DESCRIPTION'].unique())
for idx, description in enumerate(descriptions):
    mappings_description[description] = idx
df['DESCRIPTION_ID'] = df['DESCRIPTION'].apply(lambda x: mappings_description[x])
df.drop('DESCRIPTION', axis=1, inplace=True)
df_to_sql(df, '{}'.format(table_name), engine, if_exists='replace')

In [28]:
values = []
for key, val in mappings_description.items():
    values.append([val, key])
descriptions_df = pd.DataFrame(data=values, columns=columns)
df_to_sql(descriptions_df, 'wdi_descriptions_footnote', engine, if_exists='replace')

### WDI Data

In [40]:
df = pd.read_sql_table('wdidata', con=engine)
df = pd.melt(df, id_vars=['Country Code', 'Indicator Code'], value_vars=[str(val) for val in range(1960, 2019, 1)]).dropna()
df_to_sql(df, 'wdidata', engine, if_exists='replace')